In [ ]:
#################################
#
#Get average daily flow of PTT.
#
#################################
import bs4
import urllib.request as req
from urllib.parse import quote
import requests 
import pandas as pd
import os
cwd = os.getcwd()
path = cwd + "/avgDailyFlow.csv"
def getdata(pages=2000):
  count=0
  date_lst = []
  flow_lst = []
    
  url = "https://www.ptt.cc/bbs/Record/search?page=1&q=%E4%B8%8A%E7%AB%99%E4%BA%BA%E6%AC%A1%E7%B5%B1%E8%A8%88"

  n=0

  while True:
    print("------ <第"+str(n+1)+"頁資料>------")
 
    request=req.Request(url,headers={
        "cookie":"over18=1",
        "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"
    })
    
    try:
      with req.urlopen(request) as response:
        data=response.read().decode("utf-8")
      root=bs4.BeautifulSoup(data,"html.parser")
      titles=root.select("div.title")
    except:
      print(False)
      count+=1
      if count == 200:
        print(url)
        break
      else:
        continue
    
    #Get daily flow
    for title in titles:
        try:
          url = "https://www.ptt.cc"+title.a["href"]

          my_headers = {"cookie":"over18=1",
          "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"
          }
          response2 = requests.get(url, headers = my_headers)
          soup = bs4.BeautifulSoup(response2.text,"html.parser")
          header = soup.find_all("span", {"class": "hl"})
          date = header[0].text[11:-1].split("/")
          date = ":".join(date)
          avgDailyFlow = header[-1].text.strip()
          date_lst.append(date)
          flow_lst.append(avgDailyFlow)
        except Exception as e:
          print(e)
          print(url)
          print("Error happened")
          continue
    
    #Next page
    try:
      nextlink=root.find("a",string="‹ 上頁")
      url=str("https://www.ptt.cc"+nextlink["href"]) 
    except:
      print(url)
      break
        
    if n == pages-1:
      break
    n+=1
  
  df = pd.DataFrame(list(zip(date_lst, flow_lst)),columns=["date","avgDailyFlow"])
  df.to_csv(path, index=False,encoding='utf-8-sig')
  return

getdata(230)